# Predictive Analytics

In [58]:
import pandas as pd
df=pd.read_csv('https://raw.githubusercontent.com/r41ss4/rennes_da/refs/heads/main/data/merged/merged_df.csv')

C:\Users\mthig\AppData\Local\Temp\ipykernel_16940\2635943611.py:2: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('https://raw.githubusercontent.com/r41ss4/rennes_da/refs/heads/main/data/merged/merged_df.csv')


## 1. Data Splitting

In [60]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score

In [61]:
# Target and Features
target = 'price'
X = df.drop(columns=[target])
y = df[target]

In [62]:
# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [63]:
# Preprocessing
categorical_cols = ['airline', 'source_city', 'destination_city', 'departure_time', 
                    'arrival_time', 'specific_stops', 'general_stops', 'class']
numerical_cols = ['duration']

In [64]:
categorical_transformer = OneHotEncoder(handle_unknown='ignore')
numerical_transformer = StandardScaler()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

## 2. Model Selection

In [66]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [67]:
# Linear Regression Model
lr_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

In [68]:
# Random Forest Model with Hyperparameter Tuning
rf_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])

In [69]:
rf_param_grid = {
    'regressor__n_estimators': [50, 100],
    'regressor__max_depth': [10, 20, None],
    'regressor__min_samples_split': [2, 5],
}
rf_grid_search = GridSearchCV(rf_model, rf_param_grid, cv=3, scoring='neg_mean_squared_error')

## 3. Model Training

In [71]:
# Train Linear Regression
lr_model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['duration']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['airline', 'source_city',
                                                   'destination_city',
                                                   'departure_time',
                                                   'arrival_time',
                                                   'specific_stops',
                                                   'general_stops',
                                                   'class'])])),
                ('regressor', LinearRegression())])

In [ ]:
# Train and Tune Random Forest
rf_grid_search.fit(X_train, y_train)

In [ ]:
# Best Random Forest Model
best_rf_model = rf_grid_search.best_estimator_

## 4. Model Evaluation

In [ ]:
lr_preds = lr_model.predict(X_test)
lr_preds

In [ ]:
rf_preds = best_rf_model.predict(X_test)
rf_preds

In [ ]:
lr_rmse = mean_squared_error(y_test, lr_preds, squared=False)
rf_rmse = mean_squared_error(y_test, rf_preds, squared=False)

lr_r2 = r2_score(y_test, lr_preds)
rf_r2 = r2_score(y_test, rf_preds)

print(f"Linear Regression RMSE: {lr_rmse}, R^2: {lr_r2}")
print(f"Random Forest RMSE: {rf_rmse}, R^2: {rf_r2}")
print(f"Best Random Forest Parameters: {rf_grid_search.best_params_}")

## 5. Prediction

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Scatter plot
plt.figure(figsize=(8, 6))
sns.scatterplot(x=y_test, y=rf_preds)
plt.xlabel("Actual Prices")
plt.ylabel("Predicted Prices")
plt.title("Actual vs. Predicted Prices")
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
sns.scatterplot(x=y_test, y=lr_preds)
plt.xlabel("Actual Prices")
plt.ylabel("Predicted Prices")
plt.title("Actual vs. Predicted Prices")
plt.show()